In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
import numpy as np
from collections import Counter

In [ ]:
import random

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

In [ ]:
# Imbalanced learn
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.ensemble import RUSBoostClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import BalancedBaggingClassifier

# Sklearn
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Outliers detection
from sklearn.ensemble import IsolationForest

In [ ]:
# 通过read_csv函数读取CSV文件，并将其存储到DataFrame中
train_df0 = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/DSAA5002_50015756_Gelin_Bian_final/dsaa5002_2023fall_final_datasets/Data_Q1/train/data_0.csv')
train_df1 = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/DSAA5002_50015756_Gelin_Bian_final/dsaa5002_2023fall_final_datasets/Data_Q1/train/data_1.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/DSAA5002_50015756_Gelin_Bian_final/dsaa5002_2023fall_final_datasets/Data_Q1/test/test_set.csv')


In [ ]:
train_df0.head(),train_df1.head(),test_df.head()

(           x          y          z  a  b  c  d  Is_Falling
 0  18.495860  13.766527  14.362624  0  0  0  1           0
 1  18.501072  13.827225  14.270268  0  0  1  0           0
 2  18.405950  13.868976  14.094804  1  0  0  0           0
 3  18.444572  13.910701  14.116078  0  1  0  0           0
 4  18.418470  13.933917  14.320566  0  0  0  1           0,
            x          y          z  a  b  c  d  Is_Falling
 0  19.298230  15.044445  13.195140  0  0  0  1           0
 1  19.273322  14.992369  13.298404  0  0  1  0           0
 2  19.392364  15.059970  13.320235  1  0  0  0           0
 3  19.455964  15.221361  13.124152  0  1  0  0           0
 4  18.499571  15.299723  13.280978  0  0  0  1           0,
    ID         x          y          z  a  b  c  d  Is_Falling
 0   1  6.912997  11.518698  15.471855  0  0  0  1           0
 1   2  6.936432  11.574586  15.446939  0  0  1  0           0
 2   3  6.935274  11.571790  15.437505  1  0  0  0           0
 3   4  6.886688  11.56159

In [ ]:
print(Counter(train_df0.Is_Falling))

In [ ]:
print(Counter(test_df.Is_Falling))

merge the training dataset

In [ ]:
# 创建一个空的DataFrame用于存储合并后的数据
train_df = pd.DataFrame()
path ='/content/drive/MyDrive/Colab_Notebooks/5002/DSAA5002_50015756_Gelin_Bian_final/dsaa5002_2023fall_final_datasets/Data_Q1/train/'
# 读取并合并多个CSV文件
file_list = [
    path + 'data_0.csv',
    path + 'data_1.csv',
    path + 'data_2.csv',
    path + 'data_3.csv',
    path + 'data_4.csv',
    path + 'data_5.csv',
    path + 'data_6.csv',
    path + 'data_7.csv',
    path + 'data_8.csv',
    path + 'data_9.csv',
    path + 'data_10.csv',
    path + 'data_11.csv',
    path + 'data_12.csv',
    path + 'data_13.csv',
    path + 'data_14.csv',
    path + 'data_15.csv',
    path + 'data_16.csv',
    path + 'data_17.csv',
    path + 'data_18.csv',
    path + 'data_19.csv'
]
for file in file_list:
    df = pd.read_csv(file)
    train_df = pd.concat([train_df, df])

# preprocessing

https://zhuanlan.zhihu.com/p/258506276 focal loss
https://blog.csdn.net/qq_41917697/article/details/117471604
https://zhuanlan.zhihu.com/p/404798546 PR




In [ ]:
print(train_df.describe())

                   x              y              z              a  \
count  134229.000000  134229.000000  134229.000000  134229.000000   
mean       13.189512      12.184366      11.383458       0.258551   
std         3.868037       2.705422       2.018636       0.437840   
min         4.077736       6.042775       4.600291       0.000000   
25%         9.676778      10.162940       9.975850       0.000000   
50%        13.701333      11.625925      11.344621       0.000000   
75%        16.345594      14.277065      12.933669       1.000000   
max        24.360715      19.633268      16.296521       1.000000   

                   b              c              d     Is_Falling  
count  134229.000000  134229.000000  134229.000000  134229.000000  
mean        0.257962       0.260964       0.222523       0.048969  
std         0.437515       0.439162       0.415942       0.215803  
min         0.000000       0.000000       0.000000       0.000000  
25%         0.000000       0.000000   

In [ ]:
train_df.boxplot(figsize=(10, 6))
plt.show()

In [ ]:
train_df.shape

(134229, 8)

In [ ]:
from collections import Counter
Counter(train_df['Is_Falling'])

Counter({0: 127656, 1: 6573})

In [ ]:
data0_df = train_df[train_df['Is_Falling']==0]

In [ ]:
data1_df = train_df[train_df['Is_Falling']==1]

In [ ]:
from sklearn.ensemble import IsolationForest


# 训练Isolation Forest模型
model = IsolationForest(contamination='auto')
model.fit(data0_df)

# 预测样本的离群值得分
scores = model.decision_function(data0_df)

# 根据得分确定离群值阈值
threshold = pd.Series(scores).quantile(0.05)

# 根据阈值删除离群值所在的行
data0_df = data0_df[scores >= threshold]


In [ ]:
merged_train_df = pd.concat([data0_df, data1_df])
merged_train_df.shape

(127850, 8)

In [ ]:
Counter(merged_train_df['Is_Falling'])

Counter({0: 121277, 1: 6573})

In [ ]:
x = train_df.iloc[:, :-1]
y = train_df['Is_Falling']
x_test = test_df.iloc[:, 1:-1]
y_test = test_df['Is_Falling'].to_frame(name='Is_Falling')

In [ ]:
x.info, y.info, x_test.info, y_test.info

in the question, it says ' It's important to  note that we are not only concerned about falls from great heights, but also  instances where the cat may fall from tables or similar surfaces. ' so the data around 5 could be considered.

# Random Forest

In [ ]:
# 设置类别权重
class_weights = {0: 121277 , 1: 6573 }

# 创建随机森林分类器并传递类别权重
rf = RandomForestClassifier(n_estimators=100, class_weight=class_weights, random_state=0)

# 使用训练集进行训练
rf.fit(x, y)

# 使用测试集进行预测
y_pred = rf.predict(x_test)

# 打印评估结果
print("Random Forest evaluate on validation set")
print(classification_report(y_test, y_pred))

Random Forest evaluate on validation set
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6280
           1       0.43      0.06      0.10       343

    accuracy                           0.95      6623
   macro avg       0.69      0.53      0.54      6623
weighted avg       0.92      0.95      0.93      6623



# SMOTEENN

In [ ]:
print("Before SMOTEENN")
print(Counter(train_df.Is_Falling))


Before SMOTEENN
Counter({0: 127656, 1: 6573})


In [ ]:
sme = SMOTEENN(random_state=27)
X_sme, y_sme = sme.fit_resample(x, y)

print("After SMOTEENN")
print(Counter(y_sme))



After SMOTEENN
Counter({1: 123078, 0: 118077})


In [ ]:

rf = RandomForestClassifier(n_estimators=100, random_state=0,class_weight="balanced", min_samples_leaf=5)
rf.fit(x, y)
y_pred = rf.predict(x_test) # do predict on validation set
print("Random Forest evalute on validation set")
print(classification_report(y_test, y_pred)) # Show the evaluation result

Random Forest evalute on validation set
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      6280
           1       0.46      0.24      0.31       343

    accuracy                           0.95      6623
   macro avg       0.71      0.61      0.64      6623
weighted avg       0.93      0.95      0.94      6623



# Random Undersampling

In [ ]:
# 将类别0和类别1的样本分开
class_0 = train_df[train_df['Is_Falling'] == 0]
class_1 = train_df[train_df['Is_Falling'] == 1]

In [ ]:
# 随机欠采样，从类别0中选择与类别1相同数量的样本
class_0_undersampled = class_0.sample(n=len(class_1), random_state=42)

# 合并欠采样后的类别0和类别1样本
undersampled_data = pd.concat([class_0_undersampled, class_1])

# 打乱数据
undersampled_data = undersampled_data.sample(frac=1, random_state=42)

# 欠采样后的特征和标签
undersampled_x = undersampled_data.iloc[:, :-1]
undersampled_y = undersampled_data['Is_Falling']

In [ ]:
print("After RU")
print(Counter(undersampled_y))

After RU
Counter({0: 6573, 1: 6573})


In [ ]:
class_weight = {0: 24 , 1: 1 }

In [ ]:
# Random Forest Model
rf = RandomForestClassifier(n_estimators=100, random_state=0,class_weight="balanced", max_depth=10, min_samples_leaf=5)
rf.fit(undersampled_x, undersampled_y)
y_pred = rf.predict(x_test) # do predict on validation set
print("Random Forest evalute on validation set")
print(classification_report(y_test, y_pred)) # Show the evaluation result

Random Forest evalute on validation set
              precision    recall  f1-score   support

           0       0.97      0.80      0.88      6280
           1       0.14      0.61      0.23       343

    accuracy                           0.79      6623
   macro avg       0.56      0.71      0.56      6623
weighted avg       0.93      0.79      0.85      6623



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


# SMOTE

In [ ]:
print("Before SMOTE")
print(Counter(train_df.Is_Falling))

smote = SMOTE(random_state=0)

X_smote, y_smote = smote.fit_resample(x, y)

print("After SMOTE")
print(Counter(y_smote))

Before SMOTE
Counter({0: 127656, 1: 6573})
After SMOTE
Counter({0: 127656, 1: 127656})


In [ ]:
class_weight = {0: 24 , 1: 1 }

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=0, class_weight=class_weights ,min_samples_leaf=5)
rf.fit(x, y)
y_pred = rf.predict(x_test) # do predict on validation set
print("evalute on validation set")
print(classification_report(y_test, y_pred)) # Show the evaluation result

evalute on validation set
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      6280
           1       0.47      0.27      0.34       343

    accuracy                           0.95      6623
   macro avg       0.71      0.62      0.66      6623
weighted avg       0.94      0.95      0.94      6623



# Focal loss

In [ ]:
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)

In [ ]:
random_value_1 = random.random()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


class FocalLoss(torch.nn.Module):
    """
    二分类的Focalloss alpha 固定
    """
    def __init__(self, gamma=2, alpha=0.25, reduction='elementwise_mean'):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, _input, target):
        pt = torch.sigmoid(_input)
        alpha = self.alpha
        loss = - alpha * (1 - pt) ** self.gamma * target * torch.log(pt) - \
               (1 - alpha) * pt ** self.gamma * (1 - target) * torch.log(1 - pt)
        if self.reduction == 'elementwise_mean':
            loss = torch.mean(loss)
        elif self.reduction == 'sum':
            loss = torch.sum(loss)
        return loss


In [ ]:
# 将 DataFrame 转换为 Tensor
x = torch.tensor(x.values)

y = torch.tensor(y.values)

In [ ]:
x = x.float()
y_onehot = torch.eye(2)[y.long()].float()

In [ ]:
x_sme = torch.tensor(X_sme.values).float()

In [ ]:
y_sme_onehot = torch.eye(2)[y_sme].float()

In [ ]:
unique_labels = set(undersampled_y)
print(unique_labels)

{0, 1}


In [ ]:
print(type(undersampled_x))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
undersampled_x = torch.tensor(undersampled_x.values).float()


In [ ]:
# 使用get_dummies函数进行独热编码
undersampled_y_encoded = pd.get_dummies(undersampled_y).values

In [ ]:
x_test = torch.tensor(x_test.values)
y_test = torch.tensor(y_test.values)

In [ ]:

# 创建模型和损失函数
model = nn.Sequential(
    nn.Linear(7, 16),  # 第一个隐藏层，输入维度为7，输出维度为16
    nn.ReLU(),         # 激活函数
    nn.Linear(16, 32), # 第二个隐藏层，输入维度为16，输出维度为32
    nn.ReLU(),         # 激活函数
    nn.Linear(32, 2)   # 输出层，输入维度为32，输出维度为2
)
criterion = FocalLoss(alpha=0.949, gamma=2.1)  # Focal Loss 损失函数

# 优化器
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(x)

    loss = criterion(outputs, y_onehot)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}: Loss = {loss.item()}")



In [ ]:
# 继续训练
for epoch in range(10, 20):  # 增加循环的迭代次数
    optimizer.zero_grad()
    outputs = model(x)

    loss = criterion(outputs, y_onehot)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}: Loss = {loss.item()}")

In [ ]:
model.eval()  # 将模型设置为评估模式，关闭 Dropout 和 BatchNormalization 等层的随机性

with torch.no_grad():  # 关闭梯度计算，加快推理过程
    test_outputs = model(x_test.to(torch.float32))
    test_predictions = torch.argmax(test_outputs, dim=1)  # 获取预测结果（类别预测）
print(classification_report(y_test, test_predictions.tolist()))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      6280
           1       0.64      0.96      0.77       343

    accuracy                           0.97      6623
   macro avg       0.82      0.97      0.88      6623
weighted avg       0.98      0.97      0.97      6623



In [194]:
y_test

,Is_Falling
0,0
1,0
2,0
3,0
4,0
...,...
6618,0
6619,0
6620,0
6621,0


In [190]:
test_id = test_df['ID']

In [198]:
df = pd.DataFrame({'Is_Falling': test_predictions})

In [199]:
Is_Falling_df = pd.DataFrame(test_predictions, columns=['Is_Falling'])

In [200]:
merged_df = pd.concat([test_id, Is_Falling_df], axis=1)

In [201]:
merged_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/5002/DSAA5002_50015756_Gelin_Bian_final/Q1/Q1_output.csv')

train on unsampled data

try on y_sme,x_sme

# 自动调参